## Step 1: Import Python Libraries 

In [ ]:
#Keras
from tensorflow import keras

# Import of keras model and hidden layers for CNN
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout

#Image handling libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd

#Sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import matplotlib.pyplot as plt
from matplotlib import style



## Step 2: Load Dataset 

In [ ]:
#Initialize a list of paths for images
imagepaths = []

import os
for dirname, _, filenames in os.walk('./dataset'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        imagepaths.append(path)

print(len(imagepaths))

In [ ]:
IMG_SIZE=128
X=[]
y=[]
for image in imagepaths:
    try:
        img = cv2.imread(image,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))

        X.append(np.array(img))
        if(image.startswith('./dataset/Plain')):
            y.append('Plain')
        else:
            y.append('Pothole')
    except:
        pass

## Step 3: Exploratory Data Analysis 

In [ ]:
import random as rn
fig,ax=plt.subplots(2,5)
plt.subplots_adjust(bottom=0.3, top=0.7, hspace=0)
fig.set_size_inches(15,15)

for i in range(2):
    for j in range (5):
        l=rn.randint(0,len(y))
        ax[i,j].imshow(X[l][:,:,::-1])
        ax[i,j].set_title(y[l])
        ax[i,j].set_aspect('equal')

## Step 4: Data Pre-processing 

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

X=np.array(X)

labelencoder=LabelEncoder()
Y=labelencoder.fit_transform(y)
Y=to_categorical(Y,2)
print(Y)
print(X.shape)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=5)

## Step 5: Data Augmentation 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
    )

datagen.fit(x_train)

train_generator = datagen.flow(x_train, y_train, batch_size=64, subset='training')



validation_generator = datagen.flow(x_train, y_train, batch_size=64, subset='validation')

## Step 6: Build Model 

In [ ]:
# Creates a CNN Sequential Model
model = Sequential()

model.add(Conv2D(16, (3,3), activation = 'relu', padding = 'same', input_shape=(128,128,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32, (3, 3), activation='relu', padding = 'same')) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding = 'same')) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding = 'same')) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu', padding = 'same')) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu', padding = 'same')) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu', padding = 'same')) 
model.add(MaxPooling2D((2, 2)))


model.add(Flatten())

model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))

model.add(Dense(2, activation='softmax'))

In [ ]:
print(model.summary())

## Step 7: Define Loss function and optimizer 

In [ ]:
#Defining loss function and optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Step 8: Train model 

In [ ]:
history = model.fit_generator(generator=train_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    epochs = 300,
                    workers=-1)

In [ ]:
def render_training_history(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    plt.figure(figsize=(14, 4))

    plt.subplot(1, 2, 1)
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.plot(loss, label='Training set')
    plt.plot(val_loss, label='Validation set', linestyle='--')
    plt.legend()
    plt.grid(linestyle='--', linewidth=1, alpha=0.5)

    plt.subplot(1, 2, 2)
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.plot(accuracy, label='Training set')
    plt.plot(val_accuracy, label='Validation set', linestyle='--')
    plt.legend()
    plt.grid(linestyle='--', linewidth=1, alpha=0.5)

    plt.show()

In [ ]:
render_training_history(history)

## Step 9: Evaluate Model 

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)

print('Test accuracy: {:2.2f}%'.format(accuracy*100))

In [ ]:
# Making predictions on test data
prediction = model.predict(x_test)

In [ ]:
#Transform predictions into 1D array 
y_pred = np.argmax(prediction, axis=1)

In [ ]:
y_test1=y_test.astype(int)
y_test2=[]
for i in y_test1:
    a=1
    #print(i[0],i[1])
    if(i[0]==1 and i[1]==0):
        a=0
    y_test2.append(a)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

labels = ["Plain", "Potholes"]

cm = confusion_matrix(y_test2, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Plain', 'Potholes']
print(classification_report(y_test2, y_pred, target_names=target_names))

In [ ]:
model.save('newmodel.h5')